# Description

This notebook downloads the daily stock file data from CRSP to output tables containing the following variables:
- date
- permno as unique identifier
- mcap as shares outstanding times price
- return
- intraday extreme value volatility estimate $\bar{\sigma}^{}_{i,t} = \sqrt{0.3607}(p_{i,t}^{high}-p_{i,t}^{low})^{}$ based on Parkinson (1980), where $p_{i,t}$ is the logarithm of a price


## TODO
- Same permco can have multiple permno
- Complete data resriction is too tight

## Imports

In [1]:
import wrds
import pandas as pd
import numpy as np
import datetime as dt
import sys

## Set up WRDS Connection

In [47]:
wrds_conn = wrds.Connection(wrds_username='felixbru')
#wrds_connection.create_pgpass_file()
#wrds_connection.close()

Loading library list...
Done


## Explore database

In [3]:
libraries = wrds_conn.list_libraries()
library = 'crsp'

In [4]:
library_tables = wrds_conn.list_tables(library=library)
table = 'dsf'

In [5]:
table_description = wrds_conn.describe_table(library=library, table=table)#.sort_values('name')

Approximately 96285900 rows in crsp.dsf.


## Download CRSP data

### SQL Query

EXCHCD:
- 1: NYSE
- 2: NYSE MKT
- 3: NASDAQ

SHRCD:
- 10: Ordinary common share, no special status found
- 11: Ordinary common share, no special status necessary

In [6]:
sql_crsp = '''
    SELECT 
    a.permno,
    a.date,
    a.ret,
    a.shrout * a.prc as mcap,
    SQRT(0.3607) * POWER(LOG(NULLIF(a.askhi, 0)) - LOG(NULLIF(a.bidlo, 0)), 1) as vola
    
    FROM crsp.dsf as a
    
    LEFT JOIN crsp.msenames as b
    ON a.permno=b.permno
    AND b.namedt<=a.date
    AND a.date<=b.nameendt
    
    WHERE a.date BETWEEN '11/01/1999' AND '12/31/2019'
    AND b.exchcd BETWEEN 1 AND 3
    AND b.shrcd BETWEEN 10 AND 11
    '''

In [7]:
t0 = dt.datetime.today()
df_crsp_raw = wrds_conn.raw_sql(sql_crsp)
t1 = dt.datetime.today()
print('downloaded', str(dt.datetime.today()), 'in', (t1-t0).seconds, 'seconds')

downloaded 2020-04-23 10:52:29.525067 in 1378 seconds


In [9]:
df_crsp_raw.to_pickle(path='../data/external/df_crsp_raw.pkl')

In [ ]:
#df_crsp_raw = pd.read_pickle('../data/external/df_crsp_raw.pkl')

### Analytics

In [10]:
len(df_crsp_raw.permno.unique()) #6153 #3844 #1324

11395

In [11]:
#len(df_crsp_raw.permco.unique()) #6153 #3844 #1324

In [12]:
#df_crsp_raw.head(5)

In [13]:
#df_crsp_raw.describe()

In [14]:
print(sys.getsizeof(df_crsp_raw)/1e+6, 'MByte')

1614.336272 MByte


### Transform to tidy data format

In [15]:
df_crsp_tidy = df_crsp_raw.reset_index()
del df_crsp_raw

In [16]:
# edit data formats
df_crsp_tidy['date'] = pd.to_datetime(df_crsp_tidy['date'], yearfirst=True)
df_crsp_tidy[['permno']] = df_crsp_tidy[['permno']].astype(int)

In [17]:
# declare index & sort
df_crsp_tidy.set_index(['date','permno'], inplace=True)
df_crsp_tidy = df_crsp_tidy.drop(columns='index')
df_crsp_tidy = df_crsp_tidy.sort_index()

In [18]:
# rank last market cap of each year and merge to frame
s_rankcap = df_crsp_tidy['mcap']\
                .to_frame()\
                .groupby([pd.Grouper(freq='Y', level='date'), pd.Grouper(level='permno')]).last()\
                .groupby('date').rank(ascending=False)\
                .fillna(0)\
                .astype(int)
s_rankcap.columns = ['size_rank']
df_crsp_tidy = df_crsp_tidy\
                        .merge(s_rankcap, how='left', left_index=True, right_on=['date', 'permno'])\
                        .drop(columns='mcap')
# forward fill values
df_crsp_tidy['size_rank'] = df_crsp_tidy['size_rank']\
                                .groupby('permno').shift(1)\
                                .groupby('permno').ffill()#\
                                #.fillna(0)\
                                #.astype(int)
# drop initial month
drop_year = df_crsp_tidy.index.get_level_values('date').year.min()
df_crsp_tidy = df_crsp_tidy[df_crsp_tidy.index.get_level_values('date').year > drop_year]

In [19]:
df_crsp_tidy.to_pickle(path='../data/interim/df_crsp_tidy.pkl')

In [6]:
#df_crsp_tidy = pd.read_pickle('../data/interim/df_crsp_tidy.pkl')

## Download CRSP names & select assets

### SQL Query

In [20]:
sql_names = '''
    SELECT
    permno,
    comnam, 
    ticker,
    st_date,
    end_date,
    exchcd
    
    FROM crsp_a_stock.stocknames
    '''

In [21]:
t0 = dt.datetime.today()
df_crsp_names = wrds_conn.raw_sql(sql_names)
t1 = dt.datetime.today()
print('downloaded', str(dt.datetime.today()), 'in', (t1-t0).seconds, 'seconds')

downloaded 2020-04-23 11:31:25.415802 in 3 seconds


In [22]:
print(sys.getsizeof(df_crsp_names)/1e+6, 'MByte')

14.425234 MByte


### Descriptive data

In [23]:
df_crsp_names['permno'] = df_crsp_names['permno'].astype(int)
df_descriptive = df_crsp_names\
                    .groupby('permno').last()\
                    .drop(columns=['st_date','end_date'])

In [24]:
s_min_rank = df_crsp_tidy['size_rank']\
                    .groupby('permno').min()
s_min_rank.name = 'min_rank'
s_ret_missing = df_crsp_tidy['ret']\
                    .unstack()\
                    .isna()\
                    .any()
s_ret_missing.name = 'ret_missing'
s_vola_missing = df_crsp_tidy['vola']\
                    .unstack()\
                    .isna()\
                    .any()
s_vola_missing.name = 'vola_missing'

In [25]:
df_descriptive = df_descriptive\
                    .merge(s_min_rank, how='outer', left_index=True, right_index=True)\
                    .merge(s_ret_missing, how='outer', left_index=True, right_index=True)\
                    .merge(s_vola_missing, how='outer', left_index=True, right_index=True)

### Filter

In [26]:
df_descriptive.head()

,comnam,ticker,exchcd,min_rank,ret_missing,vola_missing
permno,,,,,,
10000,OPTIMUM MANUFACTURING INC,OMFGA,3.0,NaN,NaN,NaN
10001,GAS NATURAL INC,EGAS,2.0,3045.0,True,True
10002,BANCTRUST FINANCIAL GROUP INC,BTFG,3.0,2066.0,True,True
10003,GREAT COUNTRY BK ASONIA CT,GCBK,3.0,NaN,NaN,NaN
10005,WESTERN ENERGY RESOURCES INC,WERC,3.0,NaN,NaN,NaN


In [219]:
n_largest = 200
large = df_descriptive['min_rank']<=n_largest

complete_ret = df_descriptive['ret_missing']==False
include_ret = large & complete_ret

complete_vola = df_descriptive['vola_missing']==False
include_vola = large & complete_vola

include = include_ret & include_vola

In [106]:
include_permnos_ret = list(df_descriptive.index[include])
include_permnos_vola = list(df_descriptive.index[include])

In [107]:
len(include_permnos_ret), len(include_permnos_vola)

(224, 224)

## Download FF data

### SQL Query

In [116]:
sql_ff = '''
    SELECT
    date, 
    mktrf,
    smb,
    hml,
    rf,
    umd
    
    FROM ff_all.factors_daily 
    
    WHERE date BETWEEN '01/01/2000' AND '12/31/2019'
    '''

In [117]:
t0 = dt.datetime.today()
df_ff_raw = wrds_conn.raw_sql(sql_ff)
t1 = dt.datetime.today()
print('downloaded', str(dt.datetime.today()), 'in', (t1-t0).seconds, 'seconds')

downloaded 2020-04-23 20:14:07.702929 in 33 seconds


In [118]:
print(sys.getsizeof(df_ff_raw)/1e+6, 'MByte')

0.402632 MByte


### Edit

In [119]:
# edit data formats
df_ff_raw['date'] = pd.to_datetime(df_ff_raw['date'], yearfirst=True)

In [120]:
# declare index
df_ff_raw.set_index('date', inplace=True)

In [121]:
df_ff_raw.to_pickle(path='../data/external/df_ff_raw.pkl')

## Output matrices

In [224]:
df_descriptive.loc[include]\
    .drop(columns=['ret_missing', 'vola_missing'])\
    .to_csv('../data/processed/descriptive.csv')

In [109]:
rows = [i in include_permnos_vola for i in df_crsp_tidy.index.get_level_values('permno')]
vola_matrix = df_crsp_tidy['vola'][rows].unstack()

In [110]:
vola_matrix.to_csv('../data/processed/vola_matrix.csv')

In [111]:
s_rf = pd.DataFrame(index=vola_matrix.index)\
            .merge(df_ff_raw['rf'], how='left', left_index=True, right_index=True)\
            .squeeze()

In [112]:
rows = [i in include_permnos_ret for i in df_crsp_tidy.index.get_level_values('permno')]
return_matrix = df_crsp_tidy['ret'][rows]\
                    .unstack()\
                    .subtract(s_rf, axis=0)

In [113]:
return_matrix.to_csv('../data/processed/return_matrix.csv')

In [226]:
return_matrix.to_hdf('../data/processed/return_matrix.hdf', key='df')

In [228]:
return_matrix[return_matrix.index.year==2019].to_hdf('../data/processed/return_matrix_2019.hdf', key='df')

In [114]:
return_matrix[return_matrix.index.year==2019].to_csv('../data/processed/return_matrix_2019.csv')

In [214]:
vola_matrix[vola_matrix.index.year==2019].to_csv('../data/processed/vola_matrix_2019.csv')

In [217]:
return_matrix[return_matrix.index.year >= 2018].to_csv('../data/processed/return_matrix_2018-19.csv')

In [218]:
vola_matrix[vola_matrix.index.year >= 2018].to_csv('../data/processed/vola_matrix_2018-19.csv')

In [260]:
pd.Series([int(i) for i in return_matrix.index.year != pd.Series(return_matrix.index.year).shift(-1)]).to_csv('../data/processed/yearend_dates.csv')